# Análisis de Datos: Heart Disease (Enfermedad Cardiaca)

Notebook preparado para Google Colab — actividad de ML.

**Autor:** Adrian David Gonzalez Romero

**Objetivo:** Realizar EDA, preprocesamiento y comparar modelos de clasificación
para predecir la presencia de enfermedad cardíaca.


In [ ]:
# --- Configuración inicial ---
# Ejecuta en Google Colab. Si no estás en Colab, asegúrate de tener las librerías instaladas.
!pip install -q xgboost shap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')
print('Librerías cargadas correctamente.')

## 1) Carga del dataset

Usaremos la versión procesada del dataset de Heart Disease (Cleveland) desde UCI. El archivo será descargado directamente.


In [ ]:
# Descargar y cargar datos desde UCI (procesado - Cleveland)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
cols = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','target']
df = pd.read_csv(url, header=None, names=cols, na_values='?')
print('Datos cargados. Shape:', df.shape)
df.head()

## 2) Exploración de datos (EDA)
Mostraremos estadísticas descriptivas, valores faltantes y algunas visualizaciones.

In [ ]:
# Información general
df.info()
print('\nDescripción estadística:')
display(df.describe())

# Valores faltantes
print('\nValores faltantes por columna:')
print(df.isna().sum())

# Distribución de la variable objetivo
# En este dataset 'target' >0 indica presencia de enfermedad (transformaremos a 0/1)
display(df['target'].value_counts())

plt.figure(figsize=(6,4))
sns.countplot(x=(df['target']>0).astype(int))
plt.title('Distribución: presencia de enfermedad cardíaca (0=no, 1=sí)')
plt.show()

## 3) Preprocesamiento
Limpieza, imputación, codificación (si aplica) y escalado.
Transformaremos `target` a 0/1 donde 1 indica presencia de enfermedad.


In [ ]:
# Transformar target a 0/1
df['target'] = (df['target'] > 0).astype(int)

# Imputar columnas numéricas (ca, thal tienen '?', ya convertidos a NaN)
num_cols = df.select_dtypes(include=['float64','int64']).columns.tolist()
imputer = SimpleImputer(strategy='median')
df[num_cols] = imputer.fit_transform(df[num_cols])

# Revisar datos categóricos que puedan necesitar one-hot (cp, restecg, slope, thal)
df[['cp','restecg','slope','thal','ca']] = df[['cp','restecg','slope','thal','ca']].astype(int)

# One-hot para 'cp' y 'thal' y 'slope' si se desea:
df = pd.get_dummies(df, columns=['cp','thal','slope','restecg'], drop_first=True)

# Separar X y y
X = df.drop('target', axis=1)
y = df['target']

print('Shape X:', X.shape)
X.head()

In [ ]:
# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Escalado
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Conjuntos preparados: X_train', X_train.shape, 'X_test', X_test.shape)

## 4) Modelado
Entrenaremos: Regresión Logística, Random Forest y XGBoost. Se mostrarán métricas y matriz de confusión.

In [ ]:
# 4.1 Regresión Logística (baseline)
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print('Logistic Regression')
print(classification_report(y_test, y_pred_lr))
print('Accuracy:', accuracy_score(y_test,y_pred_lr))
print('ROC AUC:', roc_auc_score(y_test, lr.predict_proba(X_test_scaled)[:,1]))

In [ ]:
# 4.2 Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print('Random Forest')
print(classification_report(y_test, y_pred_rf))
print('Accuracy:', accuracy_score(y_test,y_pred_rf))
print('ROC AUC:', roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

In [ ]:
# 4.3 XGBoost
xgb_clf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print('XGBoost')
print(classification_report(y_test, y_pred_xgb))
print('Accuracy:', accuracy_score(y_test,y_pred_xgb))
print('ROC AUC:', roc_auc_score(y_test, xgb_clf.predict_proba(X_test)[:,1]))

## 5) Importancia de características y visualizaciones finales

In [ ]:
# Importancia de features con RandomForest
importances = rf.feature_importances_
feat_names = X.columns
feat_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)
display(feat_imp.head(15))

plt.figure(figsize=(8,6))
sns.barplot(x=feat_imp.head(15).values, y=feat_imp.head(15).index)
plt.title('Top 15 - Importancia de features (RandomForest)')
plt.show()

## 6) Conclusiones y recomendaciones

- Resumen de hallazgos.
- Limitaciones del dataset y del análisis.
- Recomendaciones para mejorar el modelo (más datos, validación cruzada, ajuste de hiperparámetros, uso de técnicas de balanceo si fuera necesario).

Puedes extender este notebook: ajuste de hiperparámetros con GridSearchCV, validación cruzada, uso de SHAP para interpretabilidad, pipelines de sklearn, etc.